# Peoject WIA
#### Matthieu Ré and Yaohui WANG
### Part1 Treatment of the dataset
### Variables:
1. final_sentence: the pure sentences in the dataset, no punctuations, no numbers, low letters
2. final_couple: the nouns marks in the sentence
3. list_index: the locals of the nous in the sentences

In [ ]:
f = open("SemEval2010_task8_training/TRAIN_FILE.TXT")

In [257]:
# read the train data line by line 
# store in a list
def readinFile(f):
    filelist = []
    line = f.readline()
    filelist.append(line)
    while line:
        line = f.readline()
        filelist.append(line)
    return filelist

filelist = readinFile(f)

In [258]:
# using 3 lists to store 3 elements
# list_sentence: each sentence in the list
# list_class: each class in the list
# list comment: each comment in the list
def separateFilelist(l):
    list_sentence = []
    list_class = []
    list_comment = []
    for i in range(len(l)):
        if (i + 1) % 4 == 1:
            list_sentence.append(l[i])
        if (i + 1) % 4 == 2:
            list_class.append(l[i])
        if (i + 1) % 4 == 3:
            list_comment.append(l[i])
    # to remove the '' in the end, no use
    list_sentence.remove('')
    return list_sentence,list_class,list_comment

list_sentence,list_class,list_comment = separateFilelist(filelist)

# here we delete the sentences which give us noises
# index: 212,2739,4218,4611,4783,6372

list_sentence.remove(list_sentence[212])
list_sentence.remove(list_sentence[2738])
list_sentence.remove(list_sentence[4216])
list_sentence.remove(list_sentence[4608])
list_sentence.remove(list_sentence[4779])
list_sentence.remove(list_sentence[6367])

list_class.remove(list_class[212])
list_class.remove(list_class[2738])
list_class.remove(list_class[4216])
list_class.remove(list_class[4608])
list_class.remove(list_class[4779])
list_class.remove(list_class[6367])

list_comment.remove(list_comment[212])
list_comment.remove(list_comment[2738])
list_comment.remove(list_comment[4216])
list_comment.remove(list_comment[4608])
list_comment.remove(list_comment[4779])
list_comment.remove(list_comment[6367])

In [259]:
def sizeoftrain(list_sentence,list_class,list_comment):
    print("The length of the sentences is: ",len(list_sentence))
    print("The length of the classes is: ",len(list_class))
    print("The length of the comments is: ",len(list_comment))
    
sizeoftrain(list_sentence,list_class,list_comment)

The length of the sentences is:  7994
The length of the classes is:  7994
The length of the comments is:  7994


In [260]:
from bs4 import BeautifulSoup
def getNouns(list_sentence):
    l_noun = []
    for sentence in list_sentence:
        flag = 0
        l1 = ''
        l2 = ''
        couple_noun = []
        for i in range(len(sentence)):
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '1'):
                flag = 1
            if (sentence[i] == '>') and (sentence[i-1] == '1') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '2'):
                flag = 2
            if (sentence[i] == '>') and (sentence[i-1] == '2') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0

            if flag == 1:
                l1 += sentence[i]
            if flag == 2:
                l2 += sentence[i]
        noun1_b = BeautifulSoup(l1)
        noun2_b = BeautifulSoup(l2)
        noun1 = noun1_b.get_text()
        noun2 = noun2_b.get_text()
        couple_noun.append(noun1)
        couple_noun.append(noun2)
        l_noun.append(couple_noun)
    return l_noun

l_noun = getNouns(list_sentence)

/home/sirius/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/sirius/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [261]:
# replace the space in the marked nous into #, and replace these nous in the sentences
# example: WANG Yaohui -> WANG#Yaohui

import nltk.data
import re

def replace_the_space(l_noun):
    l_new_noun = []
    for couple in l_noun:
        c = (couple[0].replace(' ','#'),couple[1].replace(' ','#'))
        l_new_noun.append(c)
    return l_new_noun

def replace_original_noun(list_sentence,l_new_noun):
    l_new_sentence = []
    for sentence,couple in zip(list_sentence,l_new_noun):
        left = couple[0].replace('#',' ')
        right = couple[1].replace('#',' ')
        sentence = re.sub("<e1>" + left + "</e1>",couple[0],sentence)
        sentence = re.sub("<e2>" + right + "</e2>",couple[1],sentence)
        l_new_sentence.append(sentence)
    return l_new_sentence

def get_the_new_sentence(list_sentence,l_noun):
    final_sentence = []
    
    l_new_noun = replace_the_space(l_noun)
    l_new_sentence = replace_original_noun(list_sentence,l_new_noun)
    
    for l in l_new_sentence:
        sentence_text = re.sub("[^a-zA-Z#-]"," ",l)
        words = sentence_text.lower().split()
        final_sentence.append(words)
    return final_sentence

def get_the_final_couple(l_noun):
    new_couple = replace_the_space(l_noun)
    final_couple = []
    for couple in new_couple:
        c = (couple[0].lower(),couple[1].lower())
        final_couple.append(c)
    return final_couple

# final sentences we need
final_sentence = get_the_new_sentence(list_sentence,l_noun)
# final marked nouns we need
final_couple = get_the_final_couple(l_noun)

In [264]:
def get_location(final_sentence,final_couple):
    list_index = []
    for couple,sentence in zip(final_couple,final_sentence):
        #print(sentence)
        #print(couple[0] + couple[1])
        noun_couple = (sentence.index(couple[0]),sentence.index(couple[1]))
        list_index.append(noun_couple)
    return list_index

# location of marked nouns in the sentences
list_index = get_location(final_sentence, final_couple)

## word2vec

In [268]:
# apply word2vec on our final sentences list
from gensim.models import word2vec
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level = logging.INFO)

num_features = 400
min_word_count = 1
num_workers = 8
context = 10
downsampling = 0.0001

print("Training model...")
model = word2vec.Word2Vec(final_sentence, workers = num_workers, size = num_features, min_count = min_word_count, window = context, sample = downsampling)

2017-01-08 16:43:06,336: INFO: collecting all words and their counts
2017-01-08 16:43:06,336: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-08 16:43:06,378: INFO: collected 19818 word types from a corpus of 136793 raw words and 7994 sentences
2017-01-08 16:43:06,379: INFO: Loading a fresh vocabulary
2017-01-08 16:43:06,390: INFO: min_count=40 retains 279 unique words (1% of original 19818, drops 19539)
2017-01-08 16:43:06,390: INFO: min_count=40 leaves 73735 word corpus (53% of original 136793, drops 63058)
2017-01-08 16:43:06,392: INFO: deleting the raw counts dictionary of 19818 items
2017-01-08 16:43:06,394: INFO: sample=0.0001 downsamples 279 most-common words
2017-01-08 16:43:06,395: INFO: downsampling leaves estimated 11028 word corpus (15.0% of prior 73735)
2017-01-08 16:43:06,396: INFO: estimated required memory for 279 words and 400 dimensions: 1032300 bytes
2017-01-08 16:43:06,399: INFO: resetting layer weights
2017-01-08 16:43:06,404: INFO: 

Training model...


2017-01-08 16:43:06,581: INFO: worker thread finished; awaiting finish of 7 more threads
2017-01-08 16:43:06,583: INFO: worker thread finished; awaiting finish of 6 more threads
2017-01-08 16:43:06,585: INFO: worker thread finished; awaiting finish of 5 more threads
2017-01-08 16:43:06,589: INFO: worker thread finished; awaiting finish of 4 more threads
2017-01-08 16:43:06,591: INFO: worker thread finished; awaiting finish of 3 more threads
2017-01-08 16:43:06,593: INFO: worker thread finished; awaiting finish of 2 more threads
2017-01-08 16:43:06,594: INFO: worker thread finished; awaiting finish of 1 more threads
2017-01-08 16:43:06,595: INFO: worker thread finished; awaiting finish of 0 more threads
2017-01-08 16:43:06,595: INFO: training on 683965 raw words (54888 effective words) took 0.2s, 300263 effective words/s
2017-01-08 16:43:06,596: WARNING: under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [269]:
model.init_sims(replace = True)

model_name = "wia"
model.save(model_name)

2017-01-08 16:43:08,659: INFO: precomputing L2-norms of word weight vectors
2017-01-08 16:43:08,662: INFO: saving Word2Vec object under wia, separately None
2017-01-08 16:43:08,662: INFO: not storing attribute syn0norm
2017-01-08 16:43:08,663: INFO: not storing attribute cum_table
2017-01-08 16:43:08,671: INFO: saved wia


In [270]:
model.doesnt_match("man waman child kitchen".split())

'kitchen'

In [271]:
model.most_similar("man")

[('that', 0.9998661279678345),
 ('we', 0.9998630285263062),
 ('their', 0.9998619556427002),
 ('to', 0.9998538494110107),
 ('and', 0.9998519420623779),
 ('so', 0.9998518228530884),
 ('i', 0.9998513460159302),
 ('from', 0.9998495578765869),
 ('it', 0.9998487830162048),
 ('had', 0.9998475313186646)]